In [91]:
import scipy
import cartoframes
import pandas as pd
import pysal
import geopandas as gpd
import networkx as nx
import numpy as np

In [3]:
cc = cartoframes.CartoContext()

In [39]:
collision_join_geom = cc.read('collision_join_geom', decode_geom=True)

In [40]:
collision_join_geom.columns

Index(['bike_trafdir', 'bikelane', 'collision_count', 'featuretyp',
       'nodeidfrom', 'nodeidto', 'nonped', 'number_park_lanes',
       'number_total_lanes', 'number_travel_lanes', 'posted_speed',
       'shape_length', 'snow_priority', 'streetcode', 'streetwidth_max',
       'sum_cyclist_injured', 'sum_cyclist_killed', 'sum_fatalities',
       'sum_injuries', 'sum_motorists_injured', 'sum_motorists_killed',
       'sum_pedestrian_injured', 'sum_pedestrian_killed', 'the_geom',
       'the_geom_str', 'trafdir', 'geometry'],
      dtype='object')

In [692]:
collision_join_geo = gpd.GeoDataFrame(collision_join_geom)

# Create neighborhood weight matrix

In [383]:
collision_join_geo[['nodeidfrom','nodeidto']] = collision_join_geo[['nodeidfrom','nodeidto']].astype('int')

In [695]:
collision_join_geo['node_a'] =  collision_join_geo[['nodeidfrom','nodeidto']].min(axis=1)
collision_join_geo['node_b'] =  collision_join_geo[['nodeidfrom','nodeidto']].max(axis=1)

In [696]:
collision_join_geo.columns

Index(['bike_trafdir', 'bikelane', 'collision_count', 'featuretyp',
       'nodeidfrom', 'nodeidto', 'nonped', 'number_park_lanes',
       'number_total_lanes', 'number_travel_lanes', 'posted_speed',
       'shape_length', 'snow_priority', 'streetcode', 'streetwidth_max',
       'sum_cyclist_injured', 'sum_cyclist_killed', 'sum_fatalities',
       'sum_injuries', 'sum_motorists_injured', 'sum_motorists_killed',
       'sum_pedestrian_injured', 'sum_pedestrian_killed', 'the_geom',
       'the_geom_str', 'trafdir', 'geometry', 'node_a', 'node_b'],
      dtype='object')

In [697]:
# Remove duplicates
collision_join_unique = collision_join_geo[~collision_join_geo.duplicated(subset=['node_a', 'node_b'], keep='first')]

In [698]:
collision_join_geo.shape

(145338, 29)

In [699]:
collision_join_unique.shape

(130472, 29)

In [700]:
# Set up dictionaries for look ups
a_b_dictionary= {}
b_a_dictionary= {}
for node_id, dest_ids in collision_join_unique[['node_a', 'node_b']].groupby('node_a'):
    a_b_dictionary[node_id]= list(dest_ids.node_b.values)
for node_id, dest_ids in collision_join_unique[['node_a', 'node_b']].groupby('node_b'):
    b_a_dictionary[node_id]= list(dest_ids.node_a.values)

In [701]:
len(a_b_dictionary)

73274

In [618]:
len(b_a_dictionary)

79920

In [627]:
# Build street network. Will
street_network = {}
for start_node, first_nodes in a_b_dictionary.items():
    for first_node in first_nodes:
        connected_lines = []
#         connected_lines.append((min((start_node, first_node)),max((start_node, first_node))))
        if first_node in a_b_dictionary.keys():
            for second_node in a_b_dictionary[first_node]:
                connected_line = (min((first_node, second_node)),max((first_node, second_node)))
                connected_lines.append(connected_line)
        elif first_node in b_a_dictionary.keys():
            for second_node in b_a_dictionary[first_node]:
                connected_line = (min((first_node, second_node)),max((first_node, second_node)))
                connected_lines.append(connected_line)
        street_network[(min((start_node, first_node)),max((start_node, first_node)))] = list(set(connected_lines))
        
for start_node, first_nodes in b_a_dictionary.items():
    for first_node in first_nodes:
        connected_lines = []
#         connected_lines.append((min((start_node, first_node)),max((start_node, first_node))))
        if first_node in b_a_dictionary.keys():
            for second_node in b_a_dictionary[first_node]:
                connected_line = (min((first_node, second_node)),max((first_node, second_node)))
                connected_lines.append(connected_line)
        elif first_node in a_b_dictionary.keys():
            for second_node in a_b_dictionary[first_node]:
                connected_line = (min((first_node, second_node)),max((first_node, second_node)))
                connected_lines.append(connected_line)
        if street_network[(min((start_node, first_node)),max((start_node, first_node)))]:
                already_connected = street_network[(min((start_node, first_node)),max((start_node, first_node)))]
                already_connected.extend(connected_lines)
                street_network[(min((start_node, first_node)),max((start_node, first_node)))] = list(set(already_connected))
        else: 
            street_network[(min((start_node, first_node)),max((start_node, first_node)))] = connected_lines

In [628]:
len(street_network)

130472

In [638]:
weights = pysal.weights.weights.W(street_network)

In [639]:
weights.histogram

[(1, 55),
 (2, 24540),
 (3, 52756),
 (4, 34321),
 (5, 14288),
 (6, 3477),
 (7, 961),
 (8, 64),
 (9, 9),
 (10, 1)]

In [640]:
weights.n

130472

In [644]:
collision_join_unique.shape

(130472, 29)

# Local Moran's I

In [647]:
# Sample data example, Local Moran's I Rate
f = pysal.open(pysal.examples.get_path("sids2.dbf"))
b = np.array(f.by_col('BIR79'))
e = np.array(f.by_col('SID79'))
w = pysal.open(pysal.examples.get_path("sids2.gal")).read()
np.random.seed(12345)
lm = pysal.esda.moran.Moran_Local_Rate(e, b, w)
lm.Is[:10]

array([-0.13452366, -1.21133985,  0.05019761,  0.06127125, -0.12627466,
        0.23497679,  0.26345855, -0.00951288, -0.01517879, -0.34513514])

In [651]:
collision_join_unique.set_index(keys=['node_a', 'node_b'], inplace=True)

In [654]:
collision_join_unique['collision_rate'] = collision_join_unique['collision_count']/collision_join_unique['shape_length']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [656]:
# Regular Local Moran's I (without rate, already normalized)
w = weights
b = collision_join_unique.shape_length
e = collision_join_unique.collision_rate
np.random.seed(12345)
lm = pysal.esda.moran.Moran_Local(e, w)

In [682]:
collision_join_unique.columns

Index(['bike_trafdir', 'bikelane', 'collision_count', 'featuretyp',
       'nodeidfrom', 'nodeidto', 'nonped', 'number_park_lanes',
       'number_total_lanes', 'number_travel_lanes', 'posted_speed',
       'shape_length', 'snow_priority', 'streetcode', 'streetwidth_max',
       'sum_cyclist_injured', 'sum_cyclist_killed', 'sum_fatalities',
       'sum_injuries', 'sum_motorists_injured', 'sum_motorists_killed',
       'sum_pedestrian_injured', 'sum_pedestrian_killed', 'the_geom',
       'the_geom_str', 'trafdir', 'geometry', 'collision_rate', 'lm_p_sim',
       'lm_q'],
      dtype='object')

In [ ]:
# Local Moran's I on pedestrian injured and pedestrians killed collision rates
w = weights
e = collision_join_unique.collision_join_unique
np.random.seed(12345)
lm = pysal.esda.moran.Moran_Local(e, w)
collision_join_unique['lm_p_sim_{}'.format()] = lm.p_sim
collision_join_unique['lm_q_{}'.format()] = lm.q

In [658]:
sig = lm.p_sim<0.05

In [661]:
lm.p_sim[sig].shape

(8711,)

In [677]:
lm.q.shape

(130472,)

In [678]:
collision_join_unique['lm_p_sim'] = lm.p_sim
collision_join_unique['lm_q'] = lm.q

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [679]:
cc.write(collision_join_unique, 'localmorans_result', overwrite=True)

KeyboardInterrupt: 

In [687]:
collision_join_unique.columns

Index(['bike_trafdir', 'bikelane', 'collision_count', 'featuretyp',
       'nodeidfrom', 'nodeidto', 'nonped', 'number_park_lanes',
       'number_total_lanes', 'number_travel_lanes', 'posted_speed',
       'shape_length', 'snow_priority', 'streetcode', 'streetwidth_max',
       'sum_cyclist_injured', 'sum_cyclist_killed', 'sum_fatalities',
       'sum_injuries', 'sum_motorists_injured', 'sum_motorists_killed',
       'sum_pedestrian_injured', 'sum_pedestrian_killed', 'the_geom',
       'the_geom_str', 'trafdir', 'geometry', 'collision_rate', 'lm_p_sim',
       'lm_q'],
      dtype='object')

In [ ]:
collision_join_unique

In [691]:
cc.map(layers=[cartoframes.QueryLayer('select * from localmorans_result where lm_p_sim < 0.05', color='collision_rate')])

# OSMNX scrach

In [230]:
import osmnx as ox
%matplotlib inline

nyc = ox.graph_from_place('New York City, New York, USA', network_type='drive')
# ox.plot_graph(ox.project_graph(nyc))

In [225]:
nyc_adj = nx.adj_matrix(nyc)

In [231]:
ox.save_graph_shapefile(nyc, filename='nyc-network-shape')

In [38]:
# Number of segments without collisions
y[y == 0].shape

(73177,)

In [39]:
# Number of segments with collisions
y[y > 0].shape

(46129,)

In [41]:
data.columns

Index(['bike_lane', 'bike_trafd', 'borocode', 'collision_count', 'date_creat',
       'date_modif', 'density_collision', 'density_cyclist_injured',
       'density_cyclist_killed', 'density_motorist_injured',
       'density_motorist_killed', 'density_pedestrian_injured',
       'density_pedestrian_killed', 'density_persons_injured',
       'density_persons_killed', 'frm_lvl_co', 'full_stree', 'index',
       'l_blkfc_id', 'l_high_hn', 'l_low_hn', 'l_zip', 'physicalid',
       'post_direc', 'post_modif', 'post_type', 'pre_direct', 'pre_modifi',
       'pre_type', 'r_blkfc_id', 'r_high_hn', 'r_low_hn', 'r_zip', 'rw_type',
       'shape_leng', 'snow_pri', 'st_label', 'st_name', 'st_width', 'status',
       'sum_cyclists_injured', 'sum_cyclists_killed', 'sum_fatalities',
       'sum_injuries', 'sum_motorists_injured', 'sum_motorists_killed',
       'sum_pedestrian_injured', 'sum_pedestrian_killed', 'the_geom',
       'the_geom_str', 'time_creat', 'time_modif', 'to_lvl_co', 'trafdir'],
   

In [71]:
x = data[['f''bike_lane', 'bike_trafd', 'frm_lvl_co',
        'rw_type']]

In [74]:
x.head()

,bike_lane,bike_trafd,frm_lvl_co,rw_type
cartodb_id,,,,
12324,,,13,1
13189,,,13,1
79814,3,TF,13,1
8008,,,13,10
18899,,,13,1


In [75]:
x.bike_lane.unique()

array(['', '3', '1', '4', '2', '6', '8', '7', '9', '5'], dtype=object)

# Scratch

In [218]:
adjacency_dict = {}
for row in collision_join_geo.iterrows():
    row_id = row[0]
    nodefrom = row[1]['nodeidfrom']
    nodeto = row[1]['nodeidto']
    neighbors = collision_join_geo[(collision_join_geo['nodeidfrom'] == nodefrom) | (collision_join_geo['nodeidto'] == nodeto) | (collision_join_geo['nodeidfrom'] == nodeto) | (collision_join_geo['nodeidto'] == nodefrom)]
    neighbors_id = list(neighbors.index.values)
    adjacency_dict[row_id] = neighbors_id

KeyboardInterrupt: 

In [268]:
lines = ["1 2",
         "2 3",
         "3 4"]
G = nx.parse_edgelist(lines, nodetype = int)
G.nodes()

NodeView((1, 2, 3, 4))

In [269]:
collision_join_geo.columns

Index(['bike_trafdir', 'bikelane', 'collision_count', 'featuretyp',
       'nodeidfrom', 'nodeidto', 'nonped', 'number_park_lanes',
       'number_total_lanes', 'number_travel_lanes', 'posted_speed',
       'shape_length', 'snow_priority', 'streetcode', 'streetwidth_max',
       'sum_cyclist_injured', 'sum_cyclist_killed', 'sum_fatalities',
       'sum_injuries', 'sum_motorists_injured', 'sum_motorists_killed',
       'sum_pedestrian_injured', 'sum_pedestrian_killed', 'the_geom',
       'the_geom_str', 'trafdir', 'geometry'],
      dtype='object')

In [317]:
x = roads.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
vals = [','.join(ele.split()) for ele in x]

In [318]:
vals[:3]

['9007738,0015363,0,118.434705',
 '0082632,0083439,0,263.795343',
 '0007993,0007994,0,237.912203']

In [319]:
G = nx.parse_edgelist(vals, delimiter=',', nodetype = int, data=(('collision_count',float),(('shape_length', float))))

In [320]:
G.number_of_edges()

130472

In [321]:
G.number_of_nodes()

94627

In [322]:
adj = nx.adjacency_matrix(G)

In [323]:
adj.shape

(94627, 94627)

In [324]:
adj.todense()

matrix([[0, 1, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [325]:
sparse_lobster = pysal.weights.WSP(nx.adj_matrix(G))
dense_lobster = sparse_lobster.to_W()

In [326]:
dense_lobster.histogram

[(1, 4062), (2, 41734), (3, 22608), (4, 25614), (5, 527), (6, 75), (7, 7)]

In [327]:
dense_lobster.n

94627

In [338]:
collision_join_geo_2 = collision_join_geo.buffer(0.1)
wQ = pysal.weights.Queen.from_dataframe(collision_join_geo_2)

KeyError: 'geometry'